# Calling Tools

Tool calling is a common feature of LLM applications. A [tool call](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.ToolCall.html#langchain_core.messages.tool.ToolCall) represents a call to a specific tool, and includes a name, arguments dict, and (optionally) an identifier. The arguments dict is structured `{argument_name: argument_value}`.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools. For instance, given a search engine tool, an LLM might handle a query by first issuing a call to the search engine. The system calling the LLM can receive the tool call, execute it, and return the output to the LLM to inform its response. LangChain includes a suite of [built-in tools](https://python.langchain.com/docs/integrations/tools/) and supports several methods for defining your own [custom tools](docs/modules/tools/custom_tools).

Providers adopt different conventions for formatting tool schemas and tool calls. For instance, Anthropic returns tool calls as parsed structures within a larger content block:
```
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```
whereas OpenAI separates tool calls into a distinct parameter, with arguments as JSON strings:
```
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```
LangChain implements standard interfaces for defining tools, passing them to LLMs, and representing tool calls.

## Passing tools to LLMs

Chat models supporting tool calling features implement a `.bind_tools` method, which receives a list of LangChain [tool objects](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool) and binds them to the chat model in its expected format. Subsequent invocations of the chat model will include tool schemas in its calls to the LLM.

For example, if we define some custom tools:

In [1]:
from langchain.tools import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [get_word_length, magic_function]

we can bind them to chat models as follows:

In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

llm_anthropic = ChatAnthropic(
    model="claude-3-opus-20240229",
    temperature=0,
).bind_tools(tools)

llm_openai = ChatOpenAI(
    model="gpt-4-turbo-2024-04-09",
    temperature=0,
).bind_tools(tools)

## Tool calls

If tool calls are included in a LLM response, they are attached to the corresponding [message](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage) or [message chunk](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) as a list of [tool call](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.ToolCall.html#langchain_core.messages.tool.ToolCall) objects in the `.tool_calls` attribute. A `ToolCall` is a typed dict that includes a tool name, dict of argument values, and (optionally) an identifier. Messages with no tool calls default to an empty list for this attribute.

Example:

In [4]:
query = (
    "how long is the word chrysanthemum? "
    "Also, what is the value of magic_function(3)?"
)

In [5]:
llm_anthropic.invoke(query).tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'chrysanthemum'},
  'id': 'toolu_01J2Ph5ruNo4vAYoSg4mf8t8'},
 {'name': 'magic_function',
  'args': {'input': 3},
  'id': 'toolu_01XkZpimZsjZtuEVRnM8JDEt'}]

In [6]:
llm_openai.invoke(query).tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'chrysanthemum'},
  'id': 'call_tdd4aenF3EyCRl9t61HH6J56'},
 {'name': 'magic_function',
  'args': {'input': 3},
  'id': 'call_yiao3eVlh6TFB8vZPDydP5ev'}]

The `.tool_calls` attribute should contain valid tool calls. Note that on occasion, model providers may output malformed tool calls (e.g., arguments that are not valid JSON). When parsing fails in these cases, instances of [InvalidToolCall](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.InvalidToolCall.html#langchain_core.messages.tool.InvalidToolCall) are populated in the `.invalid_tool_calls` attribute. An `InvalidToolCall` can have a name, string arguments, identifier, and error message.

### Streaming

When tools are called in a streaming context, [message chunks](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) will be populated with [tool call chunk](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.ToolCallChunk.html#langchain_core.messages.tool.ToolCallChunk) objects in a list via the `.tool_call_chunks` attribute. A `ToolCallChunk` includes optional string fields for the tool `name`, `args`, and `id`, and includes an optional integer field `index` that can be used to join chunks together. Fields are optional because portions of a tool call may be streamed across different chunks (e.g., a chunk that includes a substring of the arguments may have null values for the tool name and id).

Because message chunks inherit from their parent message class, an [AIMessageChunk](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) with tool call chunks will also include `.tool_calls` and `.invalid_tool_calls` fields. These fields are parsed best-effort from the message's tool call chunks.

Example:

In [7]:
async for chunk in llm_openai.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'get_word_length', 'args': '', 'id': 'call_D53nh457zwyII8EhMu5ELw72', 'index': 0}]
[{'name': None, 'args': '{"wo', 'id': None, 'index': 0}]
[{'name': None, 'args': 'rd": ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"chrys', 'id': None, 'index': 0}]
[{'name': None, 'args': 'anth', 'id': None, 'index': 0}]
[{'name': None, 'args': 'emum"', 'id': None, 'index': 0}]
[{'name': None, 'args': '}', 'id': None, 'index': 0}]
[{'name': 'magic_function', 'args': '', 'id': 'call_xCuQPAUsMA47wCQez18QGxIn', 'index': 1}]
[{'name': None, 'args': '{"in', 'id': None, 'index': 1}]
[{'name': None, 'args': 'put":', 'id': None, 'index': 1}]
[{'name': None, 'args': ' 3}', 'id': None, 'index': 1}]
[]


Note that adding message chunks will merge their corresponding tool call chunks. This is the principle by which LangChain's various [tool output parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/types/openai_tools/) support streaming.

For example, below we accumulate tool call chunks:

In [8]:
first = True
async for chunk in llm_openai.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

[]
[{'name': 'get_word_length', 'args': '', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"wo', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"word": ', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"word": "chrys', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"word": "chrysanth', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"word": "chrysanthemum"', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"word": "chrysanthemum"}', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}]
[{'name': 'get_word_length', 'args': '{"word": "chrysanthemum"}', 'id': 'call_tdd4aenF3EyCRl9t61HH6J56', 'index': 0}, {'name': 'magic_function', 'args': '', 'id': 'call_yiao3eVlh6TFB8vZPDydP5ev', 'index': 1}]
[{'name': 'get_word_length', 'args': '{"w

And below we accumulate tool calls to demonstrate partial parsing:

In [9]:
first = True
async for chunk in llm_openai.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

[]
[]
[{'name': 'get_word_length', 'args': {}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {'word': 'chrys'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {'word': 'chrysanth'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {'word': 'chrysanthemum'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {'word': 'chrysanthemum'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {'word': 'chrysanthemum'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}]
[{'name': 'get_word_length', 'args': {'word': 'chrysanthemum'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}, {'name': 'magic_function', 'args': {}, 'id': 'call_snvH5SBo0Ti0OLJLfYNNKOyU'}]
[{'name': 'get_word_length', 'args': {'word': 'chrysanthemum'}, 'id': 'call_Owpz7LKLjBEdUepp4R9J3pNK'}, {'name': 'magic_function', 'args': {}